In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d andrevitor/imagens-todas
! unzip imagens-todas.zip
from IPython.display import clear_output
clear_output()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.data import AUTOTUNE


import cv2
import numpy as np
import tensorflow as tf


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from tqdm import tqdm

x_test = 2

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.TruePositives(name='TP'),
      keras.metrics.TrueNegatives(name='TN'),
      keras.metrics.FalsePositives(name='FP'),
      keras.metrics.FalseNegatives(name='FN'),
]

def modelo_baseline():
    modelo_baseline = keras.Sequential(
        [
            keras.Input(shape=INPUT_SHAPE),
            keras.layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(2, activation="softmax"),
        ]
    )
    return modelo_baseline

def modelo_principal():
    modelo = keras.models.Sequential(
        [
            keras.layers.Conv2D(
                32, 7, activation="relu", padding="same", input_shape=INPUT_SHAPE
            ),
            # keras.layers.BatchNormalization(),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return modelo

def modelo_rascunho():
    model = keras.Sequential([
            keras.Input(shape=INPUT_SHAPE),
            keras.layers.Conv2D(1, kernel_size=(1, 1), activation="relu"),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(2,activation="softmax"),
    ])
    return model

def graph_train_val(history, name):
    """
    This function plots the accuracy and loss of the train and validation
    It also saves the graph locally, as png.

    Parameters
    ----------
        history : history
            result of the model.fit() function from the keras/tf library
        name : str
            name of the file

    Results
    -------
        None
    """

    history = history
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]

    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs_range = range(len(history.epoch))

    plt.figure(figsize=(8, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label="Acurácia de treino")
    plt.plot(epochs_range, val_acc, label="Acurácia de validação")
    plt.legend(loc="lower right")
    plt.title("Acurácia de treino e validação")

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label="Loss de treino")
    plt.plot(epochs_range, val_loss, label="Loss de validação")
    plt.legend(loc="upper right")
    plt.title("Loss de treino e validação")
    plt.savefig(name + ".png", bbox_inches="tight")
    plt.show()


def train_test(completo, labels, TEST_SIZE=0.2, TEST_VALID=0.2):
    path_train, path_test, label_train, label_test = train_test_split(
        completo, labels, test_size=TEST_SIZE
    )
    path_train, path_valid, label_train, label_valid = train_test_split(
        path_train, label_train, test_size=TEST_VALID
    )

    label_train = keras.utils.to_categorical(label_train)
    label_test = keras.utils.to_categorical(label_test)
    label_valid = keras.utils.to_categorical(label_valid)

    return path_train, path_valid, path_test, label_train, label_valid, label_test


def balance(label_train):
    # calculate the total number of training images in each class and
    # initialize a dictionary to store the class weights
    # trainLabels = keras.utils.to_categorical(label_train)
    classTotals = label_train.sum(axis=0)
    classWeight = {}
    for i in range(0, len(classTotals)):
        classWeight[i] = classTotals.max() / classTotals[i]

    return classWeight


def run_model_cv(
    model, name, BATCH_SIZE=32, LEARNING_RATE=1e-4, INPUT_SHAPE=[224, 224, 3]
):
    """
    edit:
        o model agora GERA o modelo, então recebe uma função, ñao o modelo já pronto
    Run selected model, using binary crossentropy as loss and Adam as the optmizer.
    Uses an early stopping of 5 epochs.
    Shows the accuracy/loss graphs.
    Shows the confusion matrix

    Parameters
    ----------
        model : keras.Sequential
            Model to be validated

    Results
    -------
        None
    """
    tic = time.time()
    tf.keras.backend.clear_session()
    num_classes = 2
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        metrics=METRICS,
    )
    my_callbacks = [
        keras.callbacks.EarlyStopping(
            patience=5,
            monitor="val_accuracy",
            restore_best_weights=True,
            min_delta=1e-3,
        ),
        keras.callbacks.CSVLogger(name + ".csv", separator=",", append=True),
    ]
    H = model.fit(
        x=train,
        validation_data=valid,
        class_weight=classWeight,
        epochs=1000,  #     steps_per_epoch=
        callbacks=my_callbacks,
        verbose=1,
    )

    results = model.evaluate(test, verbose=0)
    return results

def run_model_rascunho(model, name):
    BATCH_SIZE=32
    LEARNING_RATE=1e-4
    INPUT_SHAPE=[224,224,3]
    # tic = time.time()
    tf.keras.backend.clear_session()
    num_classes=2
    model.compile(loss="binary_crossentropy", 
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        metrics=METRICS
                 )
    my_callbacks = [keras.callbacks.EarlyStopping(patience=5,
                                                  monitor='val_accuracy',
                                                  restore_best_weights=True,
                                                  min_delta=1e-3),
                    keras.callbacks.CSVLogger(name+'.csv', separator=",", append=True),
                    ]   
    H = model.fit(
    x=train,
    validation_data=valid,
    class_weight=classWeight,
    epochs=1,#     steps_per_epoch=
    callbacks=my_callbacks,
    verbose=1)

    results = model.evaluate(test,verbose=0)
    return results
def percent(path, label, porcentagem):
    # criar df
    label = np.argmax(label, axis=1)
    dicio = {"path": path, "label": label}
    df = pd.DataFrame(dicio)
    # amostrar do df
    df = df.sample(int(len(path) * porcentagem))

    # verificar se sao os mesmos
    path = list(df.path)
    label = list(df.label)
    label = keras.utils.to_categorical(label)
    # repetir para os outros
    return path, label

def preprocessing(where='colab', mode='rgb'):
        
    def load_paths(path: str) -> list:
        """
        input:

        pega o caminho das imagens e retorna duas listas,
        controle e patologias, com os caminhos de cada imagem.
        """
        normal = []
        patologia = []
        path_normal = os.path.join(path,'Controle/')
        lst_path_normal = os.listdir(path_normal)
        lst_path_normal.sort()

        path_patologia = os.path.join(path,'Patologias/')
        lst_path_patologia = os.listdir(path_patologia)
        lst_path_patologia.sort()   


        for x in lst_path_normal:
            normal.append(path_normal + x)
        for x in lst_path_patologia:
            patologia.append(path_patologia + x)

        for i in patologia:
            if i.endswith('TIF'):
                print(i)
                patologia.remove(i)

        for i in normal:
            if i.endswith('TIF'):
                print(i)
                normal.remove(i)

        for i in patologia:
            if i.endswith('TIF'):
                print(i)
                patologia.remove(i)
        for i in normal:
            if i.endswith('TIF'):
                print(i)
                normal.remove(i)
        
        return normal, patologia

    def create_labels(normal, patologia):
        labels = len(normal) * [0] + len(patologia) * [1]  # [0,0,0,...,1,1,1]
        return labels

    def all_images(normal, patologia):
        completo = normal + patologia # [ '/content/drive/MyDrive/.../img.png]
        # Garantindo que não existam .tif no dataset
        for i in completo:
            if i.endswith('TIF'):
                print(i)
                completo.remove(i)
        print(len(completo))
        return completo
    if where == 'colab':
        normal_rgb, patologia_rgb = load_paths("img_originais")
        normal_hbo2, patologia_hbo2 = load_paths('hbo2')
        normal_mel, patologia_mel = load_paths('melanina')

    elif where == 'kaggle':
        normal_rgb, patologia_rgb = load_paths("../input/imagens-todas/img_originais")
        normal_hbo2, patologia_hbo2 = load_paths('../input/imagens-todas/hbo2')
        normal_mel, patologia_mel = load_paths('../input/imagens-todas/melanina')
 
    elif where == 'local':
        normal_rgb, patologia_rgb = load_paths(r"G:\Meu Drive\Desktop\_mestrado2022\imagens\img_originais")
        normal_hbo2, patologia_hbo2 = load_paths(r"G:\Meu Drive\Desktop\_mestrado2022\imagens\hbo2")
        normal_mel, patologia_mel = load_paths(r"G:\Meu Drive\Desktop\_mestrado2022\imagens\melanina")

    labels_rgb = create_labels(normal_rgb, patologia_rgb)
    labels = labels_rgb

    completo_rgb = all_images(normal_rgb, patologia_rgb)
    completo = completo_rgb
    if mode == '5ch':
        completo_hbo2 = all_images(normal_hbo2, patologia_hbo2)
        completo_mel = all_images(normal_mel, patologia_mel)
        completo = []
        for i,j,k in zip(completo_rgb, completo_hbo2, completo_mel):
            completo.append([i,j,k])
    if mode == '3ch':
        completo_hbo2 = all_images(normal_hbo2, patologia_hbo2)
        completo_mel = all_images(normal_mel, patologia_mel)
        completo = []
        for i,j,k in zip(completo_rgb, completo_hbo2, completo_mel):
            completo.append([i,j,k])

    
    return completo, labels
    



def data_load(mode='rgb'):
    INPUT_SHAPE = (224, 224)
    def preprocess(img_path):
            image = tf.io.read_file(img_path)
            image = tf.image.decode_jpeg(image, channels=3)
            # as imgs de hbo2 e melanina já tem o formato 224
            # Transformando em apenas uma dimensão:
            image = tf.image.rgb_to_grayscale(image)
            image = image / 255 # Não sei se é necessário, mas vamos deixar
            return image

    @tf.function
    def augment(image, label):
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_flip_left_right(image)
        image = tf.image.resize(image, INPUT_SHAPE)
        return (image, label)

    def load_images(imagePath, label):
        image = tf.io.read_file(imagePath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, INPUT_SHAPE)
        image = image / 255
        return (image, label)

    def load_images_train(imagePath, label):
        image = tf.io.read_file(imagePath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = image / 255
        return (image, label)

    def load_images_5ch(imagePath, label):
        # Preprocessamento da imagem rgb
        image = tf.io.read_file(imagePath[0])
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, INPUT_SHAPE)
        image = image / 255

        # Preprocessamento da imagem hbo2
        img_hbo2 = preprocess(imagePath[1])
        img_mel = preprocess(imagePath[2])
        image = tf.concat([image, img_hbo2, img_mel], -1)

        return (image, label)  # ((224,224,5),(1)

    def load_images_train_5ch(imagePath, label):
        image = tf.io.read_file(imagePath[0])
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224,224))
        image = image / 255

        # Preprocessamento da imagem hbo2
        img_hbo2 = preprocess(imagePath[1])
        img_mel = preprocess(imagePath[2])
        image = tf.concat([image, img_hbo2, img_mel], -1)

        print(np.shape(image))
        return (image, label)
    
    train = tf.data.Dataset.from_tensor_slices((path_train, label_train))
    
    def load_images_3ch(imagePath, label):
        # Preprocessamento da imagem rgb
        image = tf.io.read_file(imagePath[0])
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, INPUT_SHAPE)
        image = tf.image.rgb_to_grayscale(image)
        image = image / 255

        # Preprocessamento da imagem hbo2
        img_hbo2 = preprocess(imagePath[1])
        img_mel = preprocess(imagePath[2])
        image = tf.concat([image, img_hbo2, img_mel], -1)

        return (image, label)  # ((224,224,3),(1)
    
    def load_images_train_3ch(imagePath, label):
        image = tf.io.read_file(imagePath[0])
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224,224))
        image = tf.image.rgb_to_grayscale(image)
        image = image / 255

        # Preprocessamento da imagem hbo2
        img_hbo2 = preprocess(imagePath[1])
        img_mel = preprocess(imagePath[2])
        image = tf.concat([image, img_hbo2, img_mel], -1)

        print(np.shape(image))
        return (image, label)

    if mode == '5ch':
        train = (
        train.map(load_images_train_5ch, num_parallel_calls=AUTOTUNE)
        .map(augment, num_parallel_calls=AUTOTUNE)
        .cache().batch(64).prefetch(AUTOTUNE)
        )
        test = tf.data.Dataset.from_tensor_slices((path_test, label_test))
        test = (
            test.map(load_images_5ch, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )

        valid = tf.data.Dataset.from_tensor_slices((path_valid, label_valid))
        valid = (
            valid.map(load_images_5ch, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )
    elif mode == '3ch':
        train = (
        train.map(load_images_train_3ch, num_parallel_calls=AUTOTUNE)
        .map(augment, num_parallel_calls=AUTOTUNE)
        .cache().batch(64).prefetch(AUTOTUNE)
        )
        test = tf.data.Dataset.from_tensor_slices((path_test, label_test))
        test = (
            test.map(load_images_3ch, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )

        valid = tf.data.Dataset.from_tensor_slices((path_valid, label_valid))
        valid = (
            valid.map(load_images_3ch, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )
    else:
        train = (
            train.map(load_images_train, num_parallel_calls=AUTOTUNE)
            .map(augment, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )

        test = tf.data.Dataset.from_tensor_slices((path_test, label_test))
        test = (
            test.map(load_images, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )

        valid = tf.data.Dataset.from_tensor_slices((path_valid, label_valid))
        valid = (
            valid.map(load_images, num_parallel_calls=AUTOTUNE)
            .cache().batch(64).prefetch(AUTOTUNE)
        )
    return train, test, valid








def vazio___():
    """
    so para nao ter que ficar abrindo e fechando as funções
    o tempo todo no vsc
    """


In [ ]:
def run_model_cv2(name, BATCH_SIZE=32, LEARNING_RATE=1e-4, INPUT_SHAPE=[224, 224, 3]):
    num_classes = 2
    base_model = keras.applications.xception.Xception(weights="imagenet",
                                                include_top=False)

    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    avg = keras.layers.Dropout(0.5)(avg)
    output = keras.layers.Dense(num_classes, activation="sigmoid")(avg)
    model = keras.models.Model(inputs=base_model.input, outputs=output)

    ##############################################################
    for layer in base_model.layers:
        layer.trainable = False
    ##############################################################

    tic = time.time()
    tf.keras.backend.clear_session()
    num_classes = 2
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        metrics=METRICS,
    )
    my_callbacks = [
        keras.callbacks.EarlyStopping(
            patience=5,
            monitor="val_accuracy",
            restore_best_weights=True,
            min_delta=1e-3,
        ),
        keras.callbacks.CSVLogger(name + ".csv", separator=",", append=True),
    ]
    H = model.fit(
        x=train,
        validation_data=valid,
        class_weight=classWeight,
        epochs=1000,  #     steps_per_epoch=
        callbacks=my_callbacks,
        verbose=1,
    )
    results = model.evaluate(test, verbose=0)
    print(f'Top Layers: {results}')
    ##############################################################
    for layer in base_model.layers:
        layer.trainable = True
    ##############################################################
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        metrics=METRICS,
    )
    my_callbacks = [
        keras.callbacks.EarlyStopping(
            patience=5,
            monitor="val_accuracy",
            restore_best_weights=True,
            min_delta=1e-3,
        ),
        keras.callbacks.CSVLogger(name + ".csv", separator=",", append=True),
    ]
    H = model.fit(
        x=train,
        validation_data=valid,
        class_weight=classWeight,
        epochs=1000,  #     steps_per_epoch=
        callbacks=my_callbacks,
        verbose=1,
    )
    results = model.evaluate(test, verbose=0)
    print(f'All Layers: {results}')

    return results













completo, labels = preprocessing('colab','3ch') # rgb, 3ch, 5ch
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.2,0.4,0.6,0.8]:
    for i in range(15):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load('3ch') # 'rgb', '5ch', '3ch'
        run_model_cv2('pretrained_model')
        


img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
1647
1647
Teste n° 0, porcentagem = 0.2
(224, 224, 3)
Epoch 1/1000
4/4 [==============================] - 13s 2s/step - loss: 0.9868 - accuracy: 0.5779 - recall: 0.6882 - auc: 0.5838 - precision: 0.5639 - TP: 234.0000 - TN: 159.0000 - FP: 181.0000 - FN: 106.0000 - val_loss: 0.7240 - val_accuracy: 0.4615 - val_recall: 0.6923 - val_auc: 0.4386 - val_precision: 0.4737 - val_TP: 36.0000 - val_TN: 12.0000 - val_FP: 40.0000 - val_FN: 16.0000
Epoch 2/1000
4/4 [==============================] - 1s 272ms/step - loss: 1.0227 - accuracy: 0.4452 - recall: 0.5810 - auc: 0.4253 - precision: 0.4569 - TP: 122.0000 - TN: 65.0000 - FP: 145.0000 - FN: 88.0000 - val_loss: 0.7139 - val_accuracy: 0.4615 - val_recall: 0.6538 - val_auc: 0.4621 - val_precision: 0.4722 - val_TP: 34.0000 - val_TN: 14.0000 - va

In [ ]:
completo, labels = preprocessing('colab','3ch') # rgb, 3ch, 5ch
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.2,0.4,0.6,0.8]:
    for i in range(15):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load('3ch') # 'rgb', '5ch', '3ch'
        run_model_cv2('pretrained_model')

# Testes pequenos, de 1% a 10%

In [ ]:
for i in [0.01, 0.02, 0.05, 0.1]:
    print(1650*i)

16.5
33.0
82.5
165.0


In [ ]:
canais = 'rgb' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_principal(), 'modelo principal')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
Teste n° 0, porcentagem = 0.05
Epoch 1/1000
1/1 [==============================] - 9s 9s/step - loss: 0.9006 - accuracy: 0.6818 - recall: 0.6923 - auc: 0.6919 - precision: 0.6923 - TP: 36.0000 - TN: 36.0000 - FP: 16.0000 - FN: 16.0000 - val_loss: 0.6604 - val_accuracy: 0.9231 - val_recall: 0.9231 - val_auc: 0.9704 - val_precision: 0.9231 - val_TP: 12.0000 - val_TN: 12.0000 - val_FP: 1.0000 - val_FN: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 213ms/step - loss: 0.8913 - accuracy: 0.6154 - recall: 0.6154 - auc: 0.6450 - precision: 0.6154 - TP: 32.0000 - TN: 32.0000 - FP: 20.0000 - FN: 20.0000 - val_loss: 0.5988 - val_accuracy: 0.9231 - val_recall: 0.9231 - val_auc: 0.9704 - val_precision: 0.9231 - val_TP: 12.0000 - val_TN: 12.0000 - val_FP: 1.0000 - val_FN: 1.0000
Ep

In [ ]:
canais = 'rgb' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.02]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_principal(), 'modelo principal')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
Teste n° 0, porcentagem = 0.02
Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 0.9752 - accuracy: 0.7143 - recall: 0.7143 - auc: 0.7562 - precision: 0.7143 - TP: 15.0000 - TN: 15.0000 - FP: 6.0000 - FN: 6.0000 - val_loss: 0.8014 - val_accuracy: 0.4000 - val_recall: 0.4000 - val_auc: 0.6400 - val_precision: 0.4000 - val_TP: 2.0000 - val_TN: 2.0000 - val_FP: 3.0000 - val_FN: 3.0000
Epoch 2/1000
1/1 [==============================] - 0s 128ms/step - loss: 1.1010 - accuracy: 0.7143 - recall: 0.7143 - auc: 0.7710 - precision: 0.7143 - TP: 15.0000 - TN: 15.0000 - FP: 6.0000 - FN: 6.0000 - val_loss: 0.7181 - val_accuracy: 0.4000 - val_recall: 0.4000 - val_auc: 0.6400 - val_precision: 0.4000 - val_TP: 2.0000 - val_TN: 2.0000 - val_FP: 3.0000 - val_FN: 3.0000
Epoch 3/10

In [ ]:
canais = '3ch' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.02, 0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_principal(), 'modelo principal')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
1647
1647
Teste n° 0, porcentagem = 0.02
(224, 224, 3)
Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.9083 - accuracy: 0.7407 - recall: 0.7407 - auc: 0.8107 - precision: 0.7407 - TP: 20.0000 - TN: 20.0000 - FP: 7.0000 - FN: 7.0000 - val_loss: 0.6572 - val_accuracy: 0.6000 - val_recall: 0.6000 - val_auc: 0.7600 - val_precision: 0.6000 - val_TP: 3.0000 - val_TN: 3.0000 - val_FP: 2.0000 - val_FN: 2.0000
Epoch 2/1000
1/1 [==============================] - 0s 133ms/step - loss: 1.1019 - accuracy: 0.4286 - recall: 0.4286 - auc: 0.5238 - precision: 0.4286 - TP: 9.0000 - TN: 9.0000 - FP: 12.0000 - FN: 12.0000 - val_loss: 0.6262 - val_accuracy: 0.6000 - val_recall: 0.6000 - val_auc: 0.8400 - val_precision: 0.6000 - val_TP: 3.0000 - val_TN: 3.0000 - val_FP: 2.0000 - v

In [ ]:
canais = '5ch' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,5]
num_classes = 2
cv = []
for p in [0.02, 0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_principal(), 'modelo principal')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
1647
1647
Teste n° 0, porcentagem = 0.02
(224, 224, 5)
Epoch 1/1000
1/1 [==============================] - 5s 5s/step - loss: 1.1595 - accuracy: 0.6481 - recall: 0.6481 - auc: 0.7284 - precision: 0.6481 - TP: 35.0000 - TN: 35.0000 - FP: 19.0000 - FN: 19.0000 - val_loss: 0.8098 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_TP: 0.0000e+00 - val_TN: 0.0000e+00 - val_FP: 5.0000 - val_FN: 5.0000
Epoch 2/1000
1/1 [==============================] - 0s 164ms/step - loss: 1.1565 - accuracy: 0.5714 - recall: 0.5714 - auc: 0.5045 - precision: 0.5714 - TP: 12.0000 - TN: 12.0000 - FP: 9.0000 - FN: 9.0000 - val_loss: 0.6403 - val_accuracy: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000 - val_precision: 1.0000 - val_TP: 5.0000 - val_TN: 5

In [ ]:
canais = 'rgb' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.02, 0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_baseline(), 'modelo baseline')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
Teste n° 0, porcentagem = 0.02
Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 1.1008 - accuracy: 0.7963 - recall: 0.7963 - auc: 0.8767 - precision: 0.7963 - TP: 43.0000 - TN: 43.0000 - FP: 11.0000 - FN: 11.0000 - val_loss: 0.6526 - val_accuracy: 0.8000 - val_recall: 0.8000 - val_auc: 0.8800 - val_precision: 0.8000 - val_TP: 4.0000 - val_TN: 4.0000 - val_FP: 1.0000 - val_FN: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 42ms/step - loss: 1.1372 - accuracy: 0.6190 - recall: 0.6190 - auc: 0.7007 - precision: 0.6190 - TP: 13.0000 - TN: 13.0000 - FP: 8.0000 - FN: 8.0000 - val_loss: 0.6517 - val_accuracy: 0.8000 - val_recall: 0.8000 - val_auc: 0.8800 - val_precision: 0.8000 - val_TP: 4.0000 - val_TN: 4.0000 - val_FP: 1.0000 - val_FN: 1.0000
Epoch 3/1

In [ ]:
canais = '3ch' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,3]
num_classes = 2
cv = []
for p in [0.02, 0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_baseline(), 'modelo baseline')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
1647
1647
Teste n° 0, porcentagem = 0.02
(224, 224, 3)
Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.8227 - accuracy: 0.5926 - recall: 0.5926 - auc: 0.5617 - precision: 0.5926 - TP: 32.0000 - TN: 32.0000 - FP: 22.0000 - FN: 22.0000 - val_loss: 0.6953 - val_accuracy: 0.4000 - val_recall: 0.4000 - val_auc: 0.5600 - val_precision: 0.4000 - val_TP: 2.0000 - val_TN: 2.0000 - val_FP: 3.0000 - val_FN: 3.0000
Epoch 2/1000
1/1 [==============================] - 0s 50ms/step - loss: 0.8861 - accuracy: 0.4286 - recall: 0.4286 - auc: 0.2834 - precision: 0.4286 - TP: 9.0000 - TN: 9.0000 - FP: 12.0000 - FN: 12.0000 - val_loss: 0.6897 - val_accuracy: 0.4000 - val_recall: 0.4000 - val_auc: 0.5800 - val_precision: 0.4000 - val_TP: 2.0000 - val_TN: 2.0000 - val_FP: 3.0000 - 

In [ ]:
canais = '5ch' # rgb, 3ch, 5ch
completo, labels = preprocessing('colab',canais) 
INPUT_SHAPE = [224,224,5]
num_classes = 2
cv = []
for p in [0.02, 0.05, 0.1]:
    for i in range(30):
        print(f'Teste n° {i}, porcentagem = {p}')
        path_train, path_valid, path_test, label_train, label_valid, label_test = train_test(completo, labels)
        path_train, label_train = percent(path_train, label_train, p)
        path_valid, label_valid = percent(path_valid, label_valid, p)
        path_test, label_test = percent(path_test, label_test, p)
        classWeight = balance(label_train)
        train, test, valid = data_load(canais) # 'rgb', '5ch', '3ch'
        # run_model_cv2('pretrained_model')
        result = run_model_cv(modelo_baseline(), 'modelo baseline')
        print(result)

img_originais/Patologias/1ffa9459-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9468-8d87-11e8-9daf-6045cb817f5b..TIF
img_originais/Patologias/1ffa9488-8d87-11e8-9daf-6045cb817f5b..TIF
1647
1647
1647
Teste n° 0, porcentagem = 0.02
(224, 224, 5)
Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 1.2425 - accuracy: 0.6667 - recall: 0.6667 - auc: 0.6348 - precision: 0.6667 - TP: 36.0000 - TN: 36.0000 - FP: 18.0000 - FN: 18.0000 - val_loss: 1.0534 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_TP: 0.0000e+00 - val_TN: 0.0000e+00 - val_FP: 5.0000 - val_FN: 5.0000
Epoch 2/1000
1/1 [==============================] - 0s 45ms/step - loss: 1.2851 - accuracy: 0.2857 - recall: 0.2857 - auc: 0.2313 - precision: 0.2857 - TP: 6.0000 - TN: 6.0000 - FP: 15.0000 - FN: 15.0000 - val_loss: 0.9942 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_TP: 0.0